In [8]:
# Install xlsxwriter & openpyxl if you don't have it
# !pip install xlsxwriter openpyxl

import pandas as pd

PATH_DATA = '../'
# Create a sample dataframe
data = pd.read_excel(PATH_DATA+'Folios_boton_promad_2023-02-12.xlsx', dtype=str, sheet_name='Folios')
data.head()

## Tabla - Códigos de Cierre

### Creación

In [45]:
# Create a pivot table
pivot_table = pd.pivot_table(
    data[data['C2_INICIO']=='C2 PONIENTE'], 
    values='FOLIO', 
    index='CODIGO_MOTIVO_CIERRE_INSTITUCION', 
    columns=['mes_cierre', 'sin_corregir'], 
    margins=True, margins_name='TOTAL',
    aggfunc='count', fill_value=0
)

# Drop any coloumns where pair is (Any, 'Corregido')
pivot_table = pivot_table.drop(columns='Corregido', level=1)
# Sum again column TOTAL to get the total of the row
pivot_table['TOTAL'] = pivot_table.sum(axis=1)
# Rename index to Código de cierre
pivot_table.index.name = 'Código de Cierre'

In [46]:
# Generating the table content

pivot_table_final = pivot_table.loc[:'SYS'][:-1]
pivot_table_final.loc['SUB TOTAL'] = pivot_table_final.sum()
pivot_table_final = pd.concat([pivot_table_final, pivot_table.loc['SYS':]])
pivot_table = pivot_table_final

# Sum as int
pivot_table['TOTAL'] = pivot_table.drop(columns='TOTAL').sum(axis=1).astype(int)
pivot_table.reset_index(inplace=True)
print(pivot_table.columns)
pivot_table

MultiIndex([('Código de Cierre',             ''),
            (       'Diciembre', 'Sin corregir'),
            (           'Enero', 'Sin corregir'),
            (         'Febrero', 'Sin corregir'),
            (           'TOTAL',             '')],
           names=['mes_cierre', 'sin_corregir'])


mes_cierre,Código de Cierre,Diciembre,Enero,Febrero,TOTAL
sin_corregir,,Sin corregir,Sin corregir,Sin corregir,
0,ACI,3,2,4,9
1,ASI,41,26,19,86
2,CM,0,0,0,0
3,D,41,19,3,63
4,F,17,9,9,35
5,I,2,3,1,6
6,SUB TOTAL,104,59,36,199
7,SYS,163,124,107,394
8,TOTAL,267,183,143,593


### Formato

In [47]:
# Formatting the table

# 1. Set columns in level 2 Months from level 1 in uppercase only first 3 letters
pivot_table.columns = pd.MultiIndex.from_tuples(
    [((col[1], col[0][:3].upper()) if (i+1<len(pivot_table.columns) and i>0) else (col[0],col[1])) for i, col in enumerate(pivot_table.columns)]
)

# 2. Rename columns in level 0 where 'Sin Corregir' to 'Sin Reclasificar'
pivot_table.columns = pd.MultiIndex.from_tuples(
    [('Sin Reclasificar', col[1]) if col[0]=='Sin corregir' else col for col in pivot_table.columns]
)
pivot_table

Código de Cierre Sin Reclasificar           TOTAL
                                DIC  ENE  FEB      
0              ACI                3    2    4     9
1              ASI               41   26   19    86
2               CM                0    0    0     0
3                D               41   19    3    63
4                F               17    9    9    35
5                I                2    3    1     6
6        SUB TOTAL              104   59   36   199
7              SYS              163  124  107   394
8            TOTAL              267  183  143   593

In [49]:
pivot_table.index

RangeIndex(start=0, stop=9, step=1)

In [67]:

# # Write the pivot table to an Excel file
writer = pd.ExcelWriter('pivot_table.xlsx', engine='xlsxwriter')
pivot_table.to_excel(writer, sheet_name='Tabla Código de Cierre')

workbook = writer.book
worksheet = writer.sheets['Tabla Código de Cierre']

# Drop column A 
worksheet.set_column(0, 0, 1)
# Merge cells from A1 to A2 if A2 is not empty

# for i in range(1, len(pivot_table.index)):
# 	if pivot_table.iloc[i,1] != '':
# 		worksheet.merge_range(f'A{i+1}:A{i+2}', pivot_table.iloc[i,0], workbook.add_format({'bold': True, 'align': 'center', 'valign': 'vcenter'}))



writer.save()

C:\Users\M1ndBlast\AppData\Local\Temp\ipykernel_10984\3502774731.py:18: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
